In [1]:
from vllm import LLM, SamplingParams
import tqdm as notebook_tqdm
import json
import os
import re
from huggingface_hub import login

with open('/home/nd2794/keys.json', 'r') as file:
    keys = json.load(file)

cache_dir = '/proj/vondrick4/naveen/HF_CACHE_DIR'
hf_token = keys["HF_READ_TOKEN"]
login(token=hf_token)

os.environ['TRANSFORMERS_CACHE'] = cache_dir
os.environ['HUGGINGFACE_HUB_CACHE'] = cache_dir
os.environ['PYTORCH_TRANSFORMERS_CACHE'] = cache_dir
os.environ['PYTORCH_PRETRAINED_BERT_CACHE'] = cache_dir
os.environ['HF_HUB_CACHE'] = cache_dir

/proj/vondrick2/naveen/miniconda3/envs/tr-vllm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-06 08:50:49,741	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


## Define Utility Functions

In [2]:
def get_instruction(comp, text_event):
    comp_ = comp[:-1]

    common_start_before = 'You are give an event desciribed by a text. You have to think of exactly 10 possible events that could start before the event.\n'
    common_start_after = 'You are give an event desciribed by a text. You have to think of exactly 10 possible events that could start after the event.\n'
    common_end_before = 'You are give an event desciribed by a text. You have to think of exactly 10 possible events that could end before the event.\n'
    common_end_after = 'You are give an event desciribed by a text. You have to think of exactly 10 possible events that could end after the event.\n'

    json_instruction = """Your output should be in standard JSON format only. 
    There should be no text before or after the JSON.
    Your output format should be strictly in the following format:
    {
        possible_events: ["event-1", "event-2",.....]
    }\n\n
    """

    event_prompt = 'EVENT: {}'.format(text_event)
    
    if comp_ == 'start':
        before_instruction = common_start_before + json_instruction + event_prompt
        after_instruction = common_start_after + json_instruction + event_prompt
    elif comp_ == 'end':
        before_instruction = common_end_before + json_instruction + event_prompt
        after_instruction = common_end_after + json_instruction + event_prompt
        
    return {'before': before_instruction, 'after': after_instruction}

In [3]:
def process_llm_outs(llm_outs):
    match = re.search(r'{.*}', llm_outs, re.DOTALL)
    global unclear

    if match:
        answer_json_str = match.group(0)
        try:
            json_data = json.loads(answer_json_str)
            possible_events = json_data['possible_events']
        except:
            unclear+=1
            possible_events = ['An event is shown in the video.']
    else:
        unclear += 1
        possible_events = ['An event is shown in the video.']
        
    return possible_events

## Load Model

In [4]:
model_name = 'meta-llama/Meta-Llama-3.1-8B-Instruct'
llm = LLM(
    model=model_name, 
    tensor_parallel_size=4, 
    download_dir=cache_dir,
    dtype='bfloat16'
)

INFO 11-06 08:51:01 config.py:905] Defaulting to use mp for distributed inference
WARNING 11-06 08:51:01 arg_utils.py:957] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 11-06 08:51:01 config.py:1021] Chunked prefill is enabled with max_num_batched_tokens=512.
INFO 11-06 08:51:01 llm_engine.py:237] Initializing an LLM engine (v0.6.3.post1) with config: model='meta-llama/Meta-Llama-3.1-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3.1-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=131072, download_dir='/proj/vondrick4/naveen/HF_CACHE_DIR', load_format=LoadFormat.AUTO, tensor_parall

[W1106 08:51:04.351030894 socket.cpp:697] [c10d] The client socket cannot be initialized to connect to [::ffff:127.0.0.1]:41849 (errno: 97 - Address family not supported by protocol).
[W1106 08:51:04.653093057 socket.cpp:697] [c10d] The client socket cannot be initialized to connect to [::ffff:127.0.0.1]:41849 (errno: 97 - Address family not supported by protocol).
[W1106 08:51:04.653240387 socket.cpp:697] [c10d] The client socket cannot be initialized to connect to [::ffff:127.0.0.1]:41849 (errno: 97 - Address family not supported by protocol).


INFO 11-06 08:51:05 utils.py:1008] Found nccl from library libnccl.so.2
INFO 11-06 08:51:05 pynccl.py:63] vLLM is using nccl==2.20.5
(VllmWorkerProcess pid=1336844) (VllmWorkerProcess pid=1336842) (VllmWorkerProcess pid=1336843) INFO 11-06 08:51:05 utils.py:1008] Found nccl from library libnccl.so.2
INFO 11-06 08:51:05 utils.py:1008] Found nccl from library libnccl.so.2
INFO 11-06 08:51:05 utils.py:1008] Found nccl from library libnccl.so.2
(VllmWorkerProcess pid=1336844) (VllmWorkerProcess pid=1336842) INFO 11-06 08:51:05 pynccl.py:63] vLLM is using nccl==2.20.5
(VllmWorkerProcess pid=1336843) INFO 11-06 08:51:05 pynccl.py:63] vLLM is using nccl==2.20.5
INFO 11-06 08:51:05 pynccl.py:63] vLLM is using nccl==2.20.5


[W1106 08:51:04.894772319 socket.cpp:697] [c10d] The client socket cannot be initialized to connect to [::ffff:127.0.0.1]:41849 (errno: 97 - Address family not supported by protocol).


WARNING 11-06 08:51:05 custom_all_reduce.py:132] Custom allreduce is disabled because it's not supported on more than two PCIe-only GPUs. To silence this warning, specify disable_custom_all_reduce=True explicitly.
(VllmWorkerProcess pid=1336842) (VllmWorkerProcess pid=1336844) (VllmWorkerProcess pid=1336843) WARNING 11-06 08:51:05 custom_all_reduce.py:132] Custom allreduce is disabled because it's not supported on more than two PCIe-only GPUs. To silence this warning, specify disable_custom_all_reduce=True explicitly.
WARNING 11-06 08:51:05 custom_all_reduce.py:132] Custom allreduce is disabled because it's not supported on more than two PCIe-only GPUs. To silence this warning, specify disable_custom_all_reduce=True explicitly.
WARNING 11-06 08:51:05 custom_all_reduce.py:132] Custom allreduce is disabled because it's not supported on more than two PCIe-only GPUs. To silence this warning, specify disable_custom_all_reduce=True explicitly.
INFO 11-06 08:51:05 shm_broadcast.py:241] vLLM m

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:01,  2.79it/s]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:00<00:00,  2.97it/s]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:00<00:00,  4.17it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:01<00:00,  3.88it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:01<00:00,  3.67it/s]


INFO 11-06 08:51:07 model_runner.py:1067] Loading model weights took 3.7710 GB


(VllmWorkerProcess pid=1336843) INFO 11-06 08:51:07 model_runner.py:1067] Loading model weights took 3.7710 GB
(VllmWorkerProcess pid=1336844) INFO 11-06 08:51:07 model_runner.py:1067] Loading model weights took 3.7710 GB
(VllmWorkerProcess pid=1336842) INFO 11-06 08:51:07 model_runner.py:1067] Loading model weights took 3.7710 GB
INFO 11-06 08:51:08 distributed_gpu_executor.py:57] # GPU blocks: 76065, # CPU blocks: 8192
INFO 11-06 08:51:08 distributed_gpu_executor.py:61] Maximum concurrency for 131072 tokens per request: 9.29x
(VllmWorkerProcess pid=1336842) INFO 11-06 08:51:10 model_runner.py:1395] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
(VllmWorkerProcess pid=1336842) INFO 11-06 08:51:10 model_runner.py:1399] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_mem

In [5]:
sampling_params = SamplingParams(
    temperature=1.0,
    max_tokens=3000,
)

## Load the data

In [6]:
data = json.load(open('/proj/vondrick4/naveen/MMTRE/metadata/test_data_filtered_indexed_ultimate_11080(498).json', 'r'))

## Process the data

In [7]:
conversations = []
tracker = {}
i = 0

for sample in data:
    tracker[sample['id']] = {}
    instructions = get_instruction(sample['comp'], sample['event_2'])
    
    conversation_before = [
        {
            "role": "system",
            "content": "You are a helpful assistant"
        },
        {
            "role": "user",
            "content": instructions['before'],
        },
    ]
    conversation_after = [
        {
            "role": "system",
            "content": "You are a helpful assistant"
        },
        {
            "role": "user",
            "content": instructions['after'],
        },
    ]
    
    conversations.append(conversation_before)
    tracker[sample['id']]['before'] = i
    i+=1
    
    conversations.append(conversation_after)
    tracker[sample['id']]['after'] = i
    i+=1

## Run vLLM

In [8]:
outputs = llm.chat(
    messages=conversations,
    sampling_params=sampling_params,
    use_tqdm=True
)

Processed prompts:  13%|█████████                                                             | 2851/22160 [01:49<16:08, 19.94it/s, est. speed input: 3736.85 toks/s, output: 3287.79 toks/s]

KeyboardInterrupt: 

INFO 11-06 08:53:22 multiproc_worker_utils.py:120] Killing local vLLM worker processes


Processed prompts:  13%|█████████                                                             | 2851/22160 [02:00<16:08, 19.94it/s, est. speed input: 3736.85 toks/s, output: 3287.79 toks/s]

## Process & Write Outputs

In [ ]:
unclear = 0
processed_outputs = []
for out in outputs:
    processed_outputs.append(process_llm_outs(out.outputs[0].text))

In [ ]:
for sample in data:
    sample['possible_events_before'] = processed_outputs[tracker[sample['id']]['before']]
    sample['possible_events_after'] = processed_outputs[tracker[sample['id']]['after']]

In [ ]:
with open('/proj/vondrick4/naveen/MMTRE/metadata/zs_data/zero_shot_data_llama_3.1(temp_1.0).json', 'w') as json_file:
    json.dump(data, json_file, indent=4)

In [ ]:
processed_outputs

In [ ]:
100*unclear/len(processed_outputs)